In [2]:
import  numpy as np 

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister 
from qiskit import Aer, execute
from qiskit.quantum_info.operators import Operator
from qiskit.visualization import plot_bloch_multivector, plot_histogram


Here I will try to familiarize with matchgates and copmression. For theory please check the resources or Notability notes.

## Create  some matchgates

General form  of a matchgate:
$$ G(A,B)=\begin{pmatrix} p& 0&0&q\\ 0&w&x&0\\0&y&z&0\\r&0&0&s\\\end{pmatrix} $$

$$ A=\begin{pmatrix}p&q\\r&s\\\end{pmatrix} ; B=\begin{pmatrix}w&x\\y&z\\\end{pmatrix};  det(A)=det(B) . $$

In [3]:
# for beginning let's just  create a random matchgate.(not really random)

# We use these funcion to test if our 4x4 matrix are matchgate matrix.
def Match_gate_test(matrice):
    
    if matrice[0][1]!=0 or matrice[0][2]!=0 or matrice[1][0]!=0 or matrice[1][3]!=0 or matrice[2][0]!=0 or matrice[2][3]!=0 or matrice[3][1]!=0 or matrice[3][2]!=0 :
        print("Zero")
        return 0;
    else:
        if (matrice[1][1]*matrice[2][2]-matrice[1][2]*matrice[2][1])==(matrice[0][0]*matrice[3][3]-matrice[0][3]*matrice[3][0]):
            return 1;
        else:
            print("det(A)!= det(B)")
            return 0;


#Pauli matrix
I=np.array([[1,0],[0,1]],dtype=np.complex128)
X=np.array([[0,1],[1,0]],dtype=np.complex128)
Y=np.array([[0,-1.0j],[1.0j,0]],dtype=np.complex128)
Z=np.array([[1,0],[0,-1]],dtype=np.complex128)

#Useful 2 qubits matrix gates
CZ=np.array([[1,0,0,0],
             [0,1,0,0],
             [0,0,1,0],
             [0,0,0,-1]])
SWAP=np.array([[1,0,0,0],
              [0,0,1,0],
              [0,1,0,0],
              [0,0,0,1]])

CZSwap=np.matmul(CZ,SWAP) # see [1]


def R_z(theta):
    return np.array([[np.exp(-1j*theta/2), 0],
                    [0, np.exp(1j*theta/2)]])
def R_zI(theta):         
    return np.kron(R_z(theta),I)

def IR_z(theta):         
    return np.kron(I,R_z(theta))


print("matrice is a Matchgate matrix :",Match_gate_test(IR_z(np.pi/3)))
print("CZSwap is a Matchgate matrix:",Match_gate_test(CZSwap))


matrice is a Matchgate matrix : 1
CZSwap is a Matchgate matrix: 1


In [4]:
print(CZSwap)

[[ 1  0  0  0]
 [ 0  0  1  0]
 [ 0  1  0  0]
 [ 0  0  0 -1]]


In [3]:
def U_IRz(theta,qc,q1,q2,size):
    qc.unitary(IR_z(theta),[q1,q2],label='IRz_()')
    h=np.zeros((2*size,2*size))
    h[2*q1][2*q1+1]=theta/2
    return h
def U_RzI(theta,qc,q1,q2,size):
    qc.unitary(R_zI(theta),[q1,q2],label='Rz_()I')
    h=np.zeros((2*size,2*size))
    h[2*q2][2*q2+1]=theta/2
    return h
def U_CZSwap(qc,q1,q2,size):
    qc.unitary(CZSwap,[q1,q2],label='CZSwap')
    h=np.zeros((2*size,2*size))
    h[2*q1][2*q2+1]=-1*np.pi/4
    h[2*q1+1][2*q2]=np.pi/4
    h[2*q1][2*q1+1]=np.pi/4
    h[2*q2][2*q2+1]=np.pi/4
    return h

In [4]:
# anzatz block
def TH(theta,qc,q1,q2,size):
    h_l=[]
    h_l.append(U_CZSwap(qc,q1,q2,size))
    h_l.append(U_IRz(-1*theta,qc,q1,q2,size))
    h_l.append(U_RzI(theta+np.pi,qc,q1,q2,size))
    h_l.append(U_CZSwap(qc,q1,q2,size))
    return h_l

In [5]:
size=6
theta=np.pi
h_list=[]
q12=QuantumCircuit(size)
h_list.append(TH(theta,q12,0,1,size))
h_list.append(TH(theta,q12,1,2,size))
h_list.append(TH(theta,q12,2,3,size))
h_list.append(TH(theta,q12,4,5,size))
q12.draw()

┌─────────┐┌─────────┐┌─────────┐┌─────────┐                      »
q_0: ┤0        ├┤0        ├┤0        ├┤0        ├──────────────────────»
     │  CZSwap ││  IRz_() ││  Rz_()I ││  CZSwap │┌─────────┐┌─────────┐»
q_1: ┤1        ├┤1        ├┤1        ├┤1        ├┤0        ├┤0        ├»
     └─────────┘└─────────┘└─────────┘└─────────┘│  CZSwap ││  IRz_() │»
q_2: ────────────────────────────────────────────┤1        ├┤1        ├»
                                                 └─────────┘└─────────┘»
q_3: ──────────────────────────────────────────────────────────────────»
     ┌─────────┐┌─────────┐┌─────────┐┌─────────┐                      »
q_4: ┤0        ├┤0        ├┤0        ├┤0        ├──────────────────────»
     │  CZSwap ││  IRz_() ││  Rz_()I ││  CZSwap │                      »
q_5: ┤1        ├┤1        ├┤1        ├┤1        ├──────────────────────»
     └─────────┘└─────────┘└─────────┘└─────────┘                      »
«                                                                       
«q_0: ──────────────────────────────────────────────────────────────────
«     ┌─────────┐┌─────────┐                                            
«q_1: ┤0        ├┤0        ├────────────────────────────────────────────
«     │  Rz_()I ││  CZSwap │┌─────────┐┌─────────┐┌─────────┐┌─────────┐
«q_2: ┤1        ├┤1        ├┤0        ├┤0        ├┤0        ├┤0        ├
«     └─────────┘└─────────┘│  CZSwap ││  IRz_() ││  Rz_()I ││  CZSwap │
«q_3: ──────────────────────┤1        ├┤1        ├┤1        ├┤1        ├
«                           └─────────┘└─────────┘└─────────┘└─────────┘
«q_4: ──────────────────────────────────────────────────────────────────
«                                                                       
«q_5: ──────────────────────────────────────────────────────────────────
«

In [6]:
def U_Rz_Rz(theta,qc,q1,q2,size):
    qc.unitary(np.matmul(R_zI(-1*theta),IR_z(theta-np.pi)),[q1,q2],label='Rz(-t)Rz(t-pi)')
    h=np.zeros((2*size,2*size))
    h[2*q2][2*q2+1]=theta/2
    #qc.unitary(IR_z(theta-py),[q1,q2],label='IRz_()')
    h=np.zeros((2*size,2*size))
    h[2*q1][2*q1+1]=theta/2
    return h
def U_CZSwap(qc,q1,q2,size):
    qc.unitary(CZSwap,[q1,q2],label='CZSwap')
    h=np.zeros((2*size,2*size))
    h[2*q1][2*q2+1]=-1*np.pi/4
    h[2*q1+1][2*q2]=np.pi/4
    h[2*q1][2*q1+1]=np.pi/4
    h[2*q2][2*q2+1]=np.pi/4
    return h

In [7]:
# anzatz block
def TH_20(theta,qc,q1,q2,size):
    h_l=(U_CZSwap(qc,q1,q2,size))
    g=U_Rz_Rz(theta,qc,q1,q2,size)
    for i in range(len(h_l)):
        for j in range(len(h_l)):
            h_l[i][j]=h_l[i][j]+g[i][j]
    g=U_CZSwap(qc,q1,q2,size)
    for i in range(len(h_l)):
        for j in range(len(h_l)):
            h_l[i][j]=h_l[i][j]+g[i][j]
    g=U_IRz(np.pi,qc,q1,q2,size)
    for i in range(len(h_l)):
        for j in range(len(h_l)):
            h_l[i][j]=h_l[i][j]+g[i][j]
    return h_l

In [8]:
size=6
theta=np.pi/3
h_list=[]
q12=QuantumCircuit(size)
h_list.append(TH_20(theta,q12,0,1,size))
h_list.append(TH_20(theta,q12,1,2,size))
h_list.append(TH_20(theta,q12,2,3,size))
h_list.append(TH_20(theta,q12,3,5,size))
q12.draw()

┌─────────┐┌─────────────────┐┌─────────┐┌─────────┐           »
q_0: ┤0        ├┤0                ├┤0        ├┤0        ├───────────»
     │  CZSwap ││  Rz(-t)Rz(t-pi) ││  CZSwap ││  IRz_() │┌─────────┐»
q_1: ┤1        ├┤1                ├┤1        ├┤1        ├┤0        ├»
     └─────────┘└─────────────────┘└─────────┘└─────────┘│  CZSwap │»
q_2: ────────────────────────────────────────────────────┤1        ├»
                                                         └─────────┘»
q_3: ───────────────────────────────────────────────────────────────»
                                                                    »
q_4: ───────────────────────────────────────────────────────────────»
                                                                    »
q_5: ───────────────────────────────────────────────────────────────»
                                                                    »
«                                                                            »
«q_0: ───────────────────────────────────────────────────────────────────────»
«     ┌─────────────────┐┌─────────┐┌─────────┐                              »
«q_1: ┤0                ├┤0        ├┤0        ├──────────────────────────────»
«     │  Rz(-t)Rz(t-pi) ││  CZSwap ││  IRz_() │┌─────────┐┌─────────────────┐»
«q_2: ┤1                ├┤1        ├┤1        ├┤0        ├┤0                ├»
«     └─────────────────┘└─────────┘└─────────┘│  CZSwap ││  Rz(-t)Rz(t-pi) │»
«q_3: ─────────────────────────────────────────┤1        ├┤1                ├»
«                                              └─────────┘└─────────────────┘»
«q_4: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«q_5: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«                                                                    »
«q_0: ───────────────────────────────────────────────────────────────»
«                                                                    »
«q_1: ───────────────────────────────────────────────────────────────»
«     ┌─────────┐┌─────────┐                                         »
«q_2: ┤0        ├┤0        ├─────────────────────────────────────────»
«     │  CZSwap ││  IRz_() │┌─────────┐┌─────────────────┐┌─────────┐»
«q_3: ┤1        ├┤1        ├┤0        ├┤0                ├┤0        ├»
«     └─────────┘└─────────┘│         ││                 ││         │»
«q_4: ──────────────────────┤  CZSwap ├┤  Rz(-t)Rz(t-pi) ├┤  CZSwap ├»
«                           │         ││                 ││         │»
«q_5: ──────────────────────┤1        ├┤1                ├┤1        ├»
«                           └─────────┘└─────────────────┘└─────────┘»
«                
«q_0: ───────────
«                
«q_1: ───────────
«                
«q_2: ───────────
«     ┌─────────┐
«q_3: ┤0        ├
«     │         │
«q_4: ┤  IRz_() ├
«     │         │
«q_5: ┤1        ├
«     └─────────┘

## Expected value 

First we will play with  Z (ZZZ IIZII)  etc  observable because is easy to move on others.

In [9]:
demo=QuantumCircuit(6)
demo.x(3)
demo.h(4)
demo.h(5)
demo.cx(5,1)
demo.measure_all()
backend=Aer.get_backend('qasm_simulator')
shots=100
result = execute(demo, backend = backend, shots = shots).result()
counts = result.get_counts(demo)
print(counts)

{'011000': 38, '101010': 13, '111010': 23, '001000': 26}


In [10]:
for key in counts:
    print(key)

011000
101010
111010
001000


In [11]:
print(key[0])

0


In [12]:
# Z=|0><0|-|1><1|; ZZ=|00><00|-|01><01|-|10><10|+|11><11|   ZI=|00><00|+|01><01|-|10><10|-|11><11| IZ=|00><00|-|01><01|+|10><10|-|11><11|
# successful explanation in [4] and [5] and code is taken from [4]
def proces_counts( counts,z_index):

    z_index.sort(reverse=True) 
    new_counts = {}
    for key in counts:
        new_key = ''
        for index in z_index:
            new_key += key[-1 - index]
        if new_key in new_counts:
            new_counts[new_key] += counts[key]
        else:
            new_counts[new_key] = counts[key]

    return new_counts


def expect_z(counts,shots,z_index=[]):
    
    if len(z_index)==0:
        return 1
    else:
        z_counts=proces_counts(counts,z_index)
    print(z_counts)
    expectation=0
    for key in z_counts:
        sign=-1
        if key.count('1')%2==0:
            sign=1
        expectation= expectation+sign*z_counts[key]/shots
    return expectation

In [13]:
def measure_qc(qc,Obs):
    m_qc=qc.copy()
    m_qc.barrier()
    for i in range(len(Obs)):
        if(Obs[i]=='Z')or(Obs[i]=='I'):
            m_qc.measure(i,i)
        if(Obs[i]=='X'):
            m_qc.h(i)
            m_qc.measure(i,i)
        if(Obs[i]=='Y'):
            m_qc.rx(np.pi/2,i)
            m_qc.measure(i,i) 
    return m_qc

In [14]:
def expected(qc,Obs,shots,backend=Aer.get_backend('qasm_simulator')):
    mc=measure_qc(qc,Obs)
    counts=execute(mc,backend=backend,shots=shots).result().get_counts(mc)
    print(counts)
    z_index=[]
    for i in range (len(Obs)):
        if(Obs[i]!='I'):
            z_index.append(i)
    return expect_z(counts,shots,z_index)

In [15]:

demo=QuantumCircuit(2,2)
demo.h(1)
Oper='ZX'

In [16]:
expected(demo,Obs=Oper,shots=100) # if we do the calculus manualy we can verriffy 

{'00': 100}
{'00': 100}


1.0

In [17]:
demo.draw()

q_0: ─────
     ┌───┐
q_1: ┤ H ├
     └───┘
c: 2/═════

In [18]:
measure_qc(demo,Oper).draw()

░ ┌─┐        
q_0: ──────░─┤M├────────
     ┌───┐ ░ └╥┘┌───┐┌─┐
q_1: ┤ H ├─░──╫─┤ H ├┤M├
     └───┘ ░  ║ └───┘└╥┘
c: 2/═════════╩═══════╩═
              0       1

In [19]:
demo.draw()

q_0: ─────
     ┌───┐
q_1: ┤ H ├
     └───┘
c: 2/═════

In [20]:
demo=QuantumCircuit(6,6)
demo.x(3)
demo.h(4)
demo.h(5)
demo.cx(5,1)
demo.h(1)
demo.measure_all()
print(execute(demo,backend=Aer.get_backend('qasm_simulator'),shots=100).result().get_counts(demo))

{'001000 000000': 10, '001010 000000': 19, '011000 000000': 12, '011010 000000': 11, '101000 000000': 11, '101010 000000': 12, '111000 000000': 16, '111010 000000': 9}


In [21]:
demo.draw()

░ ┌─┐               
   q_0: ────────────────░─┤M├───────────────
             ┌───┐┌───┐ ░ └╥┘┌─┐            
   q_1: ─────┤ X ├┤ H ├─░──╫─┤M├────────────
             └─┬─┘└───┘ ░  ║ └╥┘┌─┐         
   q_2: ───────┼────────░──╫──╫─┤M├─────────
        ┌───┐  │        ░  ║  ║ └╥┘┌─┐      
   q_3: ┤ X ├──┼────────░──╫──╫──╫─┤M├──────
        ├───┤  │        ░  ║  ║  ║ └╥┘┌─┐   
   q_4: ┤ H ├──┼────────░──╫──╫──╫──╫─┤M├───
        ├───┤  │        ░  ║  ║  ║  ║ └╥┘┌─┐
   q_5: ┤ H ├──■────────░──╫──╫──╫──╫──╫─┤M├
        └───┘           ░  ║  ║  ║  ║  ║ └╥┘
   c: 6/═══════════════════╬══╬══╬══╬══╬══╬═
                           ║  ║  ║  ║  ║  ║ 
meas: 6/═══════════════════╩══╩══╩══╩══╩══╩═
                           0  1  2  3  4  5

In [22]:
import itertools

In [23]:
#Pauli set:
I=np.array([[1,0],[0,1]],dtype=np.complex128)
X=np.array([[0,1],[1,0]],dtype=np.complex128)
Y=np.array([[0,-1.0j],[1.0j,0]],dtype=np.complex128)
Z=np.array([[1,0],[0,-1]],dtype=np.complex128)
pauli=[I,X,Y,Z]
labels=['I','X','Y','Z']
indice=[0,1,2,3]
#Hilbert-Schmidt product
def HS(m1,m2):
    #input matrices m1 and m2 and return hilbert schmid  
    return(np.dot(m1.conjugate().transpose(), m2)).trace()
def decompose(O):
    size=len(O)
    nr_pauli=np.log2(size)
    norm_fact=1/(2**nr_pauli)

    elements=itertools.product(indice,repeat=int(nr_pauli))
    h_label=[]
    h=[]
    for i in elements:
        label=''
        matrice=pauli[i[0]]
        for  j in i :
            label=label+labels[indice[j]]
        for j in range(int(nr_pauli)-1):
            matrice=np.kron(matrice,pauli[i[j+1]])
        #print(matrice)
        h_label.append(label)
        h.append(norm_fact*HS(matrice,O))

    return h,h_label

In [24]:
O=np.kron(2*np.kron(I,I),Y)+np.kron(np.kron(I,I),X)+7*np.kron(np.kron(X,Y),Z)
h,h_label=decompose(O)
for i in range(len(h_label)):
    print(h_label[i])
    print(h[i])

III
0j
IIX
(1+0j)
IIY
(2+0j)
IIZ
0j
IXI
0j
IXX
0j
IXY
0j
IXZ
0j
IYI
0j
IYX
0j
IYY
0j
IYZ
0j
IZI
0j
IZX
0j
IZY
0j
IZZ
0j
XII
0j
XIX
0j
XIY
0j
XIZ
0j
XXI
0j
XXX
0j
XXY
0j
XXZ
0j
XYI
0j
XYX
0j
XYY
0j
XYZ
(7+0j)
XZI
0j
XZX
0j
XZY
0j
XZZ
0j
YII
0j
YIX
0j
YIY
0j
YIZ
0j
YXI
0j
YXX
0j
YXY
0j
YXZ
0j
YYI
0j
YYX
0j
YYY
0j
YYZ
0j
YZI
0j
YZX
0j
YZY
0j
YZZ
0j
ZII
0j
ZIX
0j
ZIY
0j
ZIZ
0j
ZXI
0j
ZXX
0j
ZXY
0j
ZXZ
0j
ZYI
0j
ZYX
0j
ZYY
0j
ZYZ
0j
ZZI
0j
ZZX
0j
ZZY
0j
ZZZ
0j


## Compress

In [25]:
#Pauli set:
I=np.array([[1,0],[0,1]],dtype=np.complex128)
X=np.array([[0,1],[1,0]],dtype=np.complex128)
Y=np.array([[0,-1.0j],[1.0j,0]],dtype=np.complex128)
Z=np.array([[1,0],[0,-1]],dtype=np.complex128)


def get_x(nrq):
    x=[]
    for i in range (nrq):
        x2=X
        x21=Y
        for k in range(i):
            x2=np.kron(Z,x2)
            x21=np.kron(Z,x21)
        for k in range(i+1,nrq):
            x2=np.kron(Z,x2)
            x21=np.kron(Z,x21)
        x.append(x2)
        x.append(x21)
    return x

In [26]:
def comutation(mat1,mat2):
    return np.matmul(mat1,mat2)-np.matmul(mat2,mat1)


def corelation(dens):
    nrq=int(np.log2(len(dens)))
    x=get_x(nrq)
    #print(x)
    C=[]
    for  i in range(2*nrq):
        li=[]
        for j in range(2*nrq):
            li.append(np.matmul(comutation(x[i],x[j]),dens).trace())
        C.append(li)
    return C

In [27]:
def new_state(dens):
    nrq=int(np.log2(len(dens)))
    ide=np.identity(2*nrq,dtype=np.complex128)
    C=corelation(dens)
    for i in range(len(C)):
        for j in range(len(C)):
            C[i][j]=1j*(C[i][j])
    ns=(1/(2*nrq))*(ide-C)
    return (1/(2*nrq))*(ide-C)

In [28]:
rho=np.identity(2**5,dtype=np.complex128)*(1/(2**5))
new=new_state(rho)
'''
print("Rho")
for i in rho:
    print(i)
'''
print("len rho: ",len(rho))
print("len new: ",len(new))

'''
print("New")
for i in new:
    print(i)
'''


len rho:  32
len new:  10


'\nprint("New")\nfor i in new:\n    print(i)\n'

In [142]:
# Compres initial state 

## Resources 

<ol>
<li>[Matchgates and classical simulation of
quantum circuits]()</li>
<li>[Matchgate and space-bounded quantum
computations are equivalent](https://arxiv.org/abs/0908.1467). </li>
<li></li>
<li>https://quantumcomputing.stackexchange.com/questions/11408/qiskit-z-expectation-value-from-counts</li>
<li>algorithm begginer</li>
</ol>